In [1]:

##YuYamaoka
##Rankエラー対策。真っ黒なマスクしかない画像を削除する。
##データセットにアクセスして、マスク画像からオブジェクトのない画像を見つけて削除する。

#path define
path_sum = ["0","1","2","3","4"]
path_sum = ["0","1","2","3","4","image"]

import os
from tqdm import tqdm
import glob
import cv2
import numpy as np
import copy

#function define
def blob_detection(mask_path,class_id: int):
    """
        opencvのブロブ検出関数の戻り値が微妙に変わってる気がするのでダウングレード推奨
        最新はcv2.__version == 4.5.1なので pip install -U opencv-python==3.4.13

        N個の物体が入っている一枚の二値画像(height, width, 1)から,
        opencvのブロブ検出関数を利用して (height, widht, N)のNチャネル画像を生成する
        各チャネルが１つの物体のマスク情報となっている

        cls_idxsは各チャネルのマスク情報が示す物体のクラスidのリストとなっている
        ※もとのブログではすべての物体がcell(id=1)だったのでcls_idxsはnp.ones(N)でよかった
    """
    mask = cv2.imread(mask_path, 0)
    _, mask = cv2.threshold(mask, 150, 255, cv2.THRESH_BINARY)

    tmp = cv2.connectedComponentsWithStats(mask)
    data = copy.deepcopy(tmp[1])

    #: 可視化するとブロブ検出関数の意味がよくわかる
    #: plt.imshow(data)
    #: plt.show()

    labels = []
    for label in np.unique(data):
        #: ラベルID==0は背景
        if label == 0:
            continue
        else:
            labels.append(label)

    if len(labels) == 0:
        #: 対象オブジェクトがない場合はNone
        #print(mask_path + "label is None!")
        return None, None
    else:
        mask = np.zeros((mask.shape)+(len(labels),), dtype=np.uint8)
        for n, label in enumerate(labels):
            mask[:, :, n] = np.uint8(data == label)
        cls_idxs = np.ones([mask.shape[-1]], dtype=np.int32) * class_id

        return mask, cls_idxs

def check_dataset(folder_path):
    files = glob.glob(folder_path + "/image/*.png")

    for img_path in tqdm(files):
        mask_exist = False
        for i in range(len(path_sum)):
            file_name = os.path.basename(img_path)
            mask_path = os.path.join(folder_path,path_sum[i],file_name)
            mask_id = i
            mask,class_id = blob_detection(mask_path,mask_id)
            if class_id is not None:
                mask_exist = True


        #全てのマスク画像にオブジェクトが確認されなかった場合、そのファイルを削除
        if(mask_exist == False):
            #print(file)
            #print("***********************")
            for j in range(len(path_sum2)):
                os.remove(os.path.join(folder_path,path_sum2[j],file_name))


ModuleNotFoundError: No module named 'cv2'

In [ ]:
#main
folders = [p for p in glob.glob("./*", recursive=True) if not os.path.isfile(p)]
for folder in folders:
    path = os.path.join(folder, "valid")
    check_dataset(path)
    path = os.path.join(folder, "train")
    check_dataset(path)